# TM10007 Assignment
Lidewij Hollestelle, Lisanne de Bruin, Lucas Piret and Amber van Teijnlingen

Group 7, GIST and non-GIST

## Load and analyse data

In [ ]:
"""
This cell performs the following tasks:
1. Imports necessary libraries and sets a random state for reproducibility.
2. Loads the dataset using the `load_data` function and separates it into features (`values`) and labels (`labels`).
3. Converts categorical labels ('GIST' and 'non-GIST') into numeric values (1 and 0).
4. Performs a Shapiro-Wilk test to count the number of features that are not normally distributed.
5. Generates a heatmap to visualize the correlation matrix of the features using Spearman's correlation.
6. Randomly selects 24 features and creates scatter plots for 12 pairs of features to analyze linearity.
"""

# Import libraries
from load_data import load_data
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from scipy.stats import shapiro
import random

# Initialize random state for reproducibility
rand_state = 42

# Load the dataset and print some information
data = load_data()
print(f'The number of samples: {len(data.index)}')
print(f'The number of columns: {len(data.columns)}')

# Seperate the labels from the values
labels = data['label']
values = data.drop(columns=['label'])

# Convert 'GIST' and 'non-GIST' to numeric values
labels = labels.map({'GIST': 1, 'non-GIST': 0})

# Perform Shapiro-Wilk test for normality on each feature
# and count the number of features that are not normally distributed
counter = 0
for feature in values.columns:
    stat, p_value = shapiro(values[feature])
    if p_value < 0.05:
        counter += 1
print(f'The number of features that are not normally distributed: {counter}')

# Heatmap for the entire correlation matrix
correlation_matrix = values.corr(method='spearman').abs()
plt.figure(figsize=(15, 15)) 
sns.heatmap(correlation_matrix, annot=False, cmap='coolwarm', fmt=".2f", cbar_kws={'label': 'Correlation Coefficient'})
plt.show()

# Randomly select 24 features for scatter plots to visualize the data
random.seed(rand_state)
random_features = random.sample(list(values.columns), 24)

# Create a 3x4 subplot
fig, axes = plt.subplots(3, 4, figsize=(20, 15))
axes = axes.flatten()

# Plot each pair of features
for i in range(0, 24, 2):
    feature_x = random_features[i]
    feature_y = random_features[i + 1]
    ax = axes[i // 2]
    ax.scatter(values[feature_x], values[feature_y], alpha=0.5)
    ax.set_xlabel(feature_x, fontsize=8)
    ax.set_ylabel(feature_y, fontsize=8)
    ax.tick_params(axis='both', which='major', labelsize=6)

plt.tight_layout()
plt.show()

: 

## Feature selection and dimension reduction

In [ ]:
"""
This cell defines two functions for preprocessing and feature selection:

1. preprocess_train_test(training_values, test_values):
    - Handles missing values by replacing them with the mean of the training data.
    - Standardizes the training and testing data using RobustScaler.
    - Removes features with low variance using VarianceThreshold.

2. statistical_selection(values, labels):
    - Fits a logistic regression model for each feature to compute its p-value.
    - Removes highly correlated features based on Spearman's correlation and p-values.
    - Returns the dataset with selected features.
"""

# Import libraries
from sklearn.feature_selection import VarianceThreshold
from sklearn.preprocessing import RobustScaler
from statsmodels import api as sm

def preprocess_train_test(training_values, test_values):
    """
    Preprocesses the training and testing datasets by handling missing values, standardizing the data, 
    and removing features with low variance.

    Args:
        training_values (pd.DataFrame): The training dataset with features.
        test_values (pd.DataFrame): The testing dataset with features.

    Returns:
        pd.DataFrame, pd.DataFrame: The preprocessed training and testing datasets.
    """

    # First we remove NaNs by averaging
    training_values = training_values.fillna(training_values.mean())
    test_values = test_values.fillna(training_values.mean())

    # Standardize the training & testing data separately
    scaler = RobustScaler()
    scaled_training_values = scaler.fit_transform(training_values)

    training_values = pd.DataFrame(scaled_training_values, columns=training_values.columns, index=training_values.index)

    scaled_test_values = scaler.transform(test_values)
    test_values = pd.DataFrame(scaled_test_values, columns=test_values.columns, index=test_values.index)

    # Remove features with zero variance
    selector = VarianceThreshold(threshold=0.3) 

    training_non_zero_var = selector.fit_transform(training_values)
    training_values = pd.DataFrame(training_non_zero_var, columns=training_values.columns[selector.get_support()], index=training_values.index)

    test_non_zero_var = selector.transform(test_values)
    test_values = pd.DataFrame(test_non_zero_var, columns=test_values.columns[selector.get_support()], index=test_values.index)

    return training_values, test_values

def statistical_selection(values, labels):
    # Now we can run the logistic regression model for each feature to get its correlation with the label
    p_values = {}

    for column in values.columns:
        logit_model = sm.Logit(labels, values[column])
        result = logit_model.fit(disp = False)

        p_values[column] = result.pvalues[column]

    # Now we remove highly correlated features
    # Make a correlation matrix to analyse the correlation between the features
    correlation_matrix = values.corr(method = 'spearman').abs()

    features_to_remove = []

    for feature1 in correlation_matrix.columns:
        for feature2 in correlation_matrix.columns:
            if feature1 != feature2 and feature1 not in features_to_remove and feature2 not in features_to_remove:
                if correlation_matrix[feature1][feature2] > 0.90:
                    # Remove the feature with the higher p-value
                    if p_values[feature1] > p_values[feature2]:
                        features_to_remove.append(feature1)
                    else:
                        features_to_remove.append(feature2)

    values = values.drop(columns=features_to_remove)
    
    # Remove the features from p_values as well
    #for feature in features_to_remove:
    #    del p_values[feature]

    # Select the top 30 features with the lowest p-values
    #sorted_features = sorted(p_values, key=p_values.get)[:30]
    #values = values[sorted_features]

    return values

## Make learning curve

In [ ]:
from sklearn.model_selection import learning_curve
import numpy as np

# Make learning
def plot_learning_curve(model, training_data, training_labels, cv, model_name):    
    train_sizes, train_scores, validation_scores = learning_curve(
        model, training_data, training_labels, cv=cv, n_jobs=-1, train_sizes=np.linspace(0.1, 1.0, 10)
    )

    # Calculate the mean and standard deviation of the scores
    train_mean = train_scores.mean(axis=1)
    validation_mean = validation_scores.mean(axis=1)
    train_std = train_scores.std(axis=1)
    validation_std = validation_scores.std(axis=1)

    # Plot the learning curve
    plt.figure(figsize=(8, 5))
    plt.plot(train_sizes, train_mean, label="Training score", color="r")
    plt.plot(train_sizes, validation_mean, label="Cross-validation score", color="g")
    plt.fill_between(train_sizes, train_mean - train_std, train_mean + train_std, color="r", alpha=0.2)
    plt.fill_between(train_sizes, validation_mean - validation_std, validation_mean + validation_std, color="g", alpha=0.2)

    # Adding labels and title
    plt.xlabel("Training Size")
    plt.ylabel("Score")
    plt.ylim(0.3, 1.01)
    plt.title(f"Learning Curve for {model_name}")
    plt.legend(loc="best")
    plt.grid(True)
    plt.show()

## Logistic regression

In [ ]:
from sklearn.linear_model import LogisticRegressionCV
import numpy as np
from sklearn.model_selection import cross_validate
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.model_selection import StratifiedKFold

def logistic_regression(training_values, training_labels):
    elastic_net = LogisticRegressionCV(
        solver='saga',
        penalty='elasticnet',
        Cs=[1, 10, 100],
        l1_ratios=[0.8],
        max_iter=10000,
        cv=5,
        n_jobs=-1
    )

    print('Start')
    elastic_net.fit(training_values, training_labels)

    #sfs = SequentialFeatureSelector(elastic_net, n_features_to_select=30, direction='forward', n_jobs=-1, scoring = 'accuracy')
    #sfs.fit(training_values, training_labels)

    #selected_indices = sfs.get_support(indices=True)
    #selected_features = training_values.iloc[:, selected_indices]
   
    #cv_scores = cross_val_score(elastic_net, selected_features, training_labels, cv=5, scoring = 'accuracy')

    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=rand_state)

    scoring = {
    'precision': 'precision',
    'recall': 'recall',
    'accuracy': 'accuracy',
    'roc_auc': 'roc_auc'
    }

    cv_scores = cross_validate(
        estimator=elastic_net,
        X=training_values,
        y=training_labels,
        cv=cv,
        scoring=scoring,
        return_train_score=False,
        n_jobs=-1
    )

    #plot_learning_curve(elastic_net, training_values, training_labels, cv, 'Logistic regression')

    selected_features = np.where(elastic_net.coef_ != 0)[0]

    print(f"{len(selected_features)} features selected out of {len(training_values.columns)}")

    training_values = training_values.iloc[:, selected_features]

    return elastic_net, cv_scores, training_values



## Random forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
import numpy as np
from sklearn.model_selection import cross_validate
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint
from sklearn.model_selection import StratifiedKFold

def train_random_forest(training_values, training_labels):
    # Initialize the Random Forest Classifier
    param_grid = {
        'n_estimators': randint(1, 100),
        'max_depth': randint(1, 30),
        'max_features': ['sqrt', 'log2'],
        'min_samples_leaf': randint(1, 10)
        }
    
    random_search = RandomizedSearchCV(RandomForestClassifier(), param_distributions=param_grid, n_iter=100, cv=5, n_jobs=-1, random_state=rand_state)

    random_search.fit(training_values, training_labels)
    print(f"Best hyperparameters: {random_search.best_params_}")

    # Return best model's cross-validation score
    best_model = random_search.best_estimator_

    #sfs = SequentialFeatureSelector(best_model, n_features_to_select=30, direction='forward', n_jobs=-1, scoring = 'accuracy')
    #sfs.fit(training_values, training_labels)

    #selected_indices = sfs.get_support(indices=True)
    #selected_features = training_values.iloc[:, selected_indices]
   
    #cv_scores = cross_val_score(best_model, selected_features, training_labels, cv=5, scoring = 'accuracy')

    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=rand_state)

    scoring = {
    'precision': 'precision',
    'recall': 'recall',
    'accuracy': 'accuracy',
    'roc_auc': 'roc_auc'
    }

    cv_scores = cross_validate(
        estimator=best_model,
        X=training_values,
        y=training_labels,
        cv=cv,
        scoring=scoring,
        return_train_score=False,
        n_jobs=-1
    )

    return best_model, cv_scores

    """
    feature_importances = best_model.feature_importances_

    feature_importance_df = pd.DataFrame({
        'Feature': training_values.columns,
        'Importance': feature_importances
    })

    feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=True)

    all_scores = []
    for i in range(1, 493):
        top_features = feature_importance_df.head(i)['Feature'].values

        data_selected = training_values[top_features]
        labels_selected = training_labels

        final_model = best_model.fit(data_selected, labels_selected)
        cv_scores = cross_val_score(final_model, data_selected, labels_selected, cv=5)
        all_scores.append(cv_scores.mean())

    # Plot the cross-validation scores
    plt.figure(figsize=(10, 6))
    plt.plot(range(1, 493), all_scores, marker='o', linestyle='-', color='b')
    plt.xlabel('Number of Top Features')
    plt.ylabel('Cross-Validation Accuracy')
    plt.title('Random Forest Number of Included Top Features')
    plt.grid(True)
    plt.show()
    """

 

## KNN-classifier

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_validate
from sklearn.metrics import make_scorer, precision_score, recall_score, roc_auc_score
from sklearn.model_selection import StratifiedKFold

def KNN(training_values, training_labels):
    param_grid = {
    'n_neighbors': list(range(1, 26)),
    'weights': ['uniform', 'distance']
    }

    random_search = GridSearchCV(KNeighborsClassifier(), param_grid, cv=5, n_jobs=-1)

    random_search.fit(training_values, training_labels)
    print(f"Best hyperparameters: {random_search.best_params_}")

    # Return best model's cross-validation score
    best_model = random_search.best_estimator_

    #sfs = SequentialFeatureSelector(best_model, n_features_to_select=30, direction='forward', n_jobs=-1, scoring = 'accuracy')
    #sfs.fit(training_values, training_labels)

    #selected_indices = sfs.get_support(indices=True)
    #selected_features = training_values.iloc[:, selected_indices]
   
    #cv_scores = cross_val_score(best_model, selected_features, training_labels, cv=5, scoring = 'accuracy')

    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=rand_state)

    scoring = {
    'precision': 'precision',
    'recall': 'recall',
    'accuracy': 'accuracy',
    'roc_auc': 'roc_auc'
    }

    cv_scores = cross_validate(
        estimator=best_model,
        X=training_values,
        y=training_labels,
        cv=cv,
        scoring=scoring,
        return_train_score=False,
        n_jobs=-1,
    )


    return best_model, cv_scores


## SVM

In [ ]:
from sklearn.svm import SVC
from sklearn.model_selection import RandomizedSearchCV, cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.model_selection import StratifiedKFold

def SVM(training_values, training_labels):
    param_grid = {
        'C': [0.1, 0.5, 1, 5, 10, 25], 
        'kernel': ['linear', 'rbf', 'poly', 'sigmoid'],
        'gamma': ['scale', 0.01, 0.1, 1],
        'degree': [2, 3, 4],
    }

    random_search = RandomizedSearchCV(SVC(), param_distributions=param_grid,n_iter=100, cv=5, n_jobs=-1, random_state=rand_state)

    random_search.fit(training_values, training_labels)
    print(f"Best hyperparameters: {random_search.best_params_}")

    # Return best model's cross-validation score
    best_model = random_search.best_estimator_
    print(best_model)

    #sfs = SequentialFeatureSelector(best_model, n_features_to_select=30, direction='forward', n_jobs=-1, scoring = 'accuracy')
    #sfs.fit(training_values, training_labels)

    #selected_indices = sfs.get_support(indices=True)
    #selected_features = training_values.iloc[:, selected_indices]
   
    #cv_scores = cross_val_score(best_model, selected_features, training_labels, cv=5, scoring = 'accuracy')

    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=rand_state)

    scoring = {
    'precision': 'precision',
    'recall': 'recall',
    'accuracy': 'accuracy',
    'roc_auc': 'roc_auc'
    }

    cv_scores = cross_validate(
        estimator=best_model,
        X=training_values,
        y=training_labels,
        cv=cv,
        scoring=scoring,
        return_train_score=False,
        n_jobs=-1,
    )

    return best_model, cv_scores

## Try the different classifiers for 5 different folds

In [ ]:
from sklearn.model_selection import StratifiedKFold

# 5-fold cross-validation setup
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=rand_state)
# Create a dataframe to store the results
cross_val_results = []

for fold, (train_index, test_index) in enumerate(skf.split(values, labels)):
    print(f"\n=== Fold {fold} ===")

    training_values = values.iloc[train_index]
    training_labels = labels[train_index]

    test_values = values.iloc[test_index]
    test_labels = labels[test_index]

    training_values, test_values = preprocess_train_test(training_values, test_values)

    #training_values = statistical_selection(training_values, training_labels)

    print(training_values.shape)

    # Now train the classifiers

    logit_model, logit_score, selected_values = logistic_regression(training_values, training_labels)
    print(f"Fold {fold} - Logistic Regression Score: {logit_score['test_accuracy'].mean()}")

    forest_model, forest_score = train_random_forest(training_values, training_labels)
    print(f"Fold {fold} - Random Forest Score: {forest_score['test_accuracy'].mean()}")

    KNN_model, knn_score = KNN(training_values, training_labels)
    print(f"Fold {fold} - KNN Score: {knn_score['test_accuracy'].mean()}")

    SVM_model, svm_score = SVM(training_values, training_labels)
    print(f"Fold {fold} - SVM Score: {svm_score['test_accuracy'].mean()}")

    # Append scores for each model
    cross_val_results.append({'Model': logit_model, 'Fold': fold, 'Precision': logit_score['test_precision'].mean(), 'Recall': logit_score['test_recall'].mean(), 'Accuracy': logit_score['test_accuracy'].mean(), 'ROC AUC': logit_score['test_roc_auc'].mean()})
    cross_val_results.append({'Model': forest_model, 'Fold': fold, 'Precision': forest_score['test_precision'].mean(), 'Recall': forest_score['test_recall'].mean(), 'Accuracy': forest_score['test_accuracy'].mean(), 'ROC AUC': forest_score['test_roc_auc'].mean()})
    cross_val_results.append({'Model': KNN_model, 'Fold': fold, 'Precision': knn_score['test_precision'].mean(), 'Recall': knn_score['test_recall'].mean(), 'Accuracy': knn_score['test_accuracy'].mean(), 'ROC AUC': knn_score['test_roc_auc'].mean()})
    cross_val_results.append({'Model': SVM_model, 'Fold': fold, 'Precision': svm_score['test_precision'].mean(), 'Recall': svm_score['test_recall'].mean(), 'Accuracy': svm_score['test_accuracy'].mean(), 'ROC AUC': svm_score['test_roc_auc'].mean()})


# Convert the results list to a dataframe
results_df = pd.DataFrame(cross_val_results)

results_df.to_csv('cross_val_results.csv', index=False)

print(results_df)